# Mastering the Pandas Library
This notebook covers detailed topics related to the Pandas library in Python, following a structured outline for practical use cases and applications.

## 1. Basics and Fundamentals
- Series: 1D & homogeneous → container for scalars - aka a column
    - If you create a Series from a dict, the keys will then become the labels. 
- DataFrame: 2D & heterogeneous → contains a collection of *series*
    - Can be created from dictionaries, lists, NumPy arrays, or external files.
    - With the `index` argument, you can name your own indexes for Series or row titles for DF

In [10]:
# Importing pandas
import pandas as pd

# Creating a Series
series = pd.Series([10, 20, 30], name='Sample Series')
print(series)

# Create a Series with custom labelled index
data = [10, 20, 30, 40]
labelled_series = pd.Series(data, index=["a", "b", "c", "d"])
print(labelled_series)

# Creating a DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35]}
df = pd.DataFrame(data)
print(df)

# Inspecting a DataFrame
print(f"# First 5 rows:\n{df.head()}\n") # tail()=last x rows
print("\nQuick One-shot Summary")
df.info() #directly outputs to console, can't do print(df.info())
print(f"# Summary of numerical columns(mean, count, std, min, max):\n{df.describe()}\n")
print(f"# (row count, column count): {df.shape}\n")
print(f"# Column Data types:\n{df.dtypes}\n")

0    10
1    20
2    30
Name: Sample Series, dtype: int64
a    10
b    20
c    30
d    40
dtype: int64
      Name  Age
0    Alice   25
1      Bob   30
2  Charlie   35
# First 5 rows:
      Name  Age
0    Alice   25
1      Bob   30
2  Charlie   35


Quick One-shot Summary
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      object
 1   Age     3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes
# Summary of numerical columns(mean, count, std, min, max):
        Age
count   3.0
mean   30.0
std     5.0
min    25.0
25%    27.5
50%    30.0
75%    32.5
max    35.0

# (row count, column count): (3, 2)

# Column Data types:
Name    object
Age      int64
dtype: object



## 2. Data I/O: Read/Write Files of Various Formats
**CSV**: `pd.read_csv()`, `DataFrame.to_csv()`
 - `index=False` tells Pandas **not to include the row index** in the exported file. By default, the DF row index is the 1st column in the output CSV

In [ ]:
df.to_csv('input.csv', index=False) #write
csv_data = pd.read_csv('output.csv') #read
print(csv_data)

**JSON**: `pd.read_json()` / `.to_json()`
- **`orient`**: Specifies the JSON format to control how data is structured during conversion between DF/Series and JSON. Possible values:
    - **`records`**: For row-based processing or compatibility with APIs.
    - **`columns`**: Default; useful for column-wise data storage.
    - **`index`**: For row-wise lookup based on indices.
    - **`split`**: Ideal for recreating DataFrame structure programmatically.
    - **`table`**: For interoperability with table-oriented frameworks or tools.

In [ ]:
# Reading and writing JSON files
df.to_json('example.json', orient='records')
json_data = pd.read_json('example.json')
print(json_data)

**Excel**: `pd.read_excel()`, `DataFrame.to_excel()`

In [ ]:
df = pd.read_excel('example.xlsx', sheet_name='Sheet1')
df.to_excel('output.xlsx', sheet_name='Sheet1', index=False)

**SQL**: `pd.read_sql()` / `.to_sql()` 

- **`pd.read_sql`**: Reads data into a DF from either an **entire table or a SQL query**, offering flexibility.
- **`pd.read_sql_query`**: **Executes SQL queries** and loads the result into a DF. **Only accepts SQL queries** as input.
- **SQL Connection**:A DB connection (e.g., using `sqlalchemy.create_engine`) is required for both reading and writing.
     - `con=engine`=DB connection

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:') #Connect to DB
# read_sql()
df = pd.read_sql('SELECT * FROM table_name', con=engine) #Read via query
df = pd.read_sql('table_name', con=engine)  # Reads entire table

# read_sql_query()
df = pd.read_sql_query('SELECT * FROM table_name', con=engine)

# Write to SQL DB
df.to_sql('output_table', engine)
df.to_sql('output_table', con=engine, if_exists='replace', index=False)

**Parquet**: `pd.read_parquet()` / `.to_parquet()`
- **Parquet** is a **columnar storage file format** that is optimized for efficient data storage and retrieval, commonly used in big data processing frameworks like Apache Spark, Hadoop, and Pandas. It is designed to handle large datasets efficiently.

In [ ]:
df = pd.read_parquet('example.parquet')
df.to_parquet('example.parquet', index=False)

**Handling Large Files**:Chunk-based reading (`chunksize`)->Specifies the number of rows to read at a time. 
- UC: File types with tabular data and are processed row-by-row

In [ ]:
with open("large_dataset.csv", "w") as f:
    f.write(csv_data)

# Read the CSV file in chunks
chunk_size = 2  # Number of rows per chunk
chunk_iterator = pd.read_csv("large_dataset.csv", chunksize=chunk_size)

# Process each chunk
for chunk in chunk_iterator:
    print(f"Processing chunk:\n{chunk}\n")
    # Example: Perform operations like filtering
    filtered_chunk = chunk[chunk['Age'] > 30]
    print(f"Filtered chunk:\n{filtered_chunk}\n")

## 3. Indexing and Selection
Access, subset and filter large datasets for specific rows/columns.


In [21]:
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'London', 'Paris']
}, index=['row1', 'row2', 'row3'])
print(df)

         Name  Age      City
row1    Alice   25  New York
row2      Bob   30    London
row3  Charlie   35     Paris


1. **Column Selection**:
    - Single column: `df['column_name']`
    - Multiple columns: `df[['col1', 'col2']]`

In [ ]:
# Column-Based Indexing
print("Single column (by label):\n", df['Name'])  
print("Multiple columns:\n", df[['Name', 'City']])

2. **Row Selection**: Return specified row(s) as a Series
    - By label: `df.loc['row_label']`
    - By position(int): `df.iloc[row_index]`

In [15]:
# Row-Based Indexing with `loc`
print("\nSingle row (by label):\n", df.loc['row1'])  
print("Multiple rows:\n", df.loc[['row1', 'row3']])  
print("Specific value (by label):", df.loc['row2', 'City']) 


Single row (by label):
 Name       Alice
Age           25
City    New York
Name: row1, dtype: object
Multiple rows:
          Name  Age      City
row1    Alice   25  New York
row3  Charlie   35     Paris
Specific value (by label): London


In [16]:
# Row-Based Indexing with `iloc`
print("\nSingle row (by position):\n", df.iloc[0])  
print("Multiple rows:\n", df.iloc[[0, 2]])  
print("Specific value (by position):", df.iloc[1, 2]) 


Single row (by position):
 Name       Alice
Age           25
City    New York
Name: row1, dtype: object
Multiple rows:
          Name  Age      City
row1    Alice   25  New York
row3  Charlie   35     Paris
Specific value (by position): London


3. **Specific Cell**:
    - By label: `df.loc['row_label', 'column_name']`
    - By position: `df.iloc[row_index, col_index]`

In [ ]:
cell_by_label = df.loc['row2', 'City']  
print("Cell by label:", cell_by_label)

cell_by_position = df.iloc[1, 2] 
print("Cell by position:", cell_by_position)

4. **Slicing**:
    - Rows: `df[2:5]` 
    - Rows/columns by labels(inclusive): `df.loc[start_label:end_label]`
    - Rows/columns by indices(exclusive): `df.iloc[start:end]` 
- **Label slicing includes the end label, index slicing excludes the end index!!**

In [18]:
row_slice = df[1:3]  # Slicing by row indices - exclusive 
print("Slicing rows (df[1:3]):\n", row_slice)

# Slicing Rows/Columns by Labels - inclusive
label_slice = df.loc['row1':'row2', 'Name':'Age']  
print("\nSlicing by labels:\n", label_slice)

# Slicing Rows/Columns by Indices
index_slice = df.iloc[0:2, 0:2]  
print("\nSlicing by indices:\n", index_slice)

Slicing rows (df[1:3]):
          Name  Age    City
row2      Bob   30  London
row3  Charlie   35   Paris

Slicing by labels:
        Name  Age
row1  Alice   25
row2    Bob   30

Slicing by indices:
        Name  Age
row1  Alice   25
row2    Bob   30


5. **Conditional/Boolean Indexing**:
- **Filter rows**: `df[df['column_name'] > value]`
    - Use **`df[...]`** for basic filtering.
- **Filter rows with `loc`**: `df.loc[df['column_name'] == 'value']`
    - Use **`df.loc[...]`** for filtering + selecting specific columns (label-based) 

In [24]:
# Filter rows (boolean) with df[...]
filtered_simple = df[df['Age'] > 30]
print("df[...] example:\n", filtered_simple)

# Multiple conditions
print(df[(df['Age'] > 25) & (df["Name"] == 'Charlie')]) 

# Filter rows (conditional) with df.loc[...]
filtered_advanced = df.loc[df['Age'] > 30, ['Name', 'City']]
print("\ndf.loc[...] example:\n", filtered_advanced)

df[...] example:
          Name  Age   City
row3  Charlie   35  Paris
         Name  Age   City
row3  Charlie   35  Paris

df.loc[...] example:
          Name   City
row3  Charlie  Paris


6. **Advanced Indexing**:
    - Select specific rows and columns: `df.loc[[row1, row2], [col1, col2]]`

**Setting and Resetting Index**:
 - `.set_index()`: Create a MultiIndex for hierarchical organization.
 - `.reset_index()`: Flatten the index back into regular columns.
 
 Set MultiIndex: set_index() creates MultiIndex 
Reset MultiIndex: reset_index() flattens the index back into regular columns.
- **Single-Level vs Multi-Level Data**: Single-level data uses a flat index for rows/columns, while multi-level (or hierarchical) data uses multiple index levels to represent relationships or groupings within the data structure.

In [ ]:
# Setting and resetting an index
df.set_index('Name', inplace=True)
print(df)
df.reset_index(inplace=True)
print(df)

**MultiIndex (hierarchical indexing)**
- **MultiIndex** is a hierarchical index structure->manage multi-D data by using multiple levels of row or column indices.
- **How to Create**: 
    - `set_index()` is the most common way to create a MultiIndex by promoting one or more columns to row indices.
    - Alternative: `pd.MultiIndex.from_tuples()`/`.from_product()`->more flexible

- `reset_index()` is used to revert the MultiIndex into columns.

- **Why Use It**: It organizes and analyzes grouped or hierarchical datasets more efficiently, enabling operations like slicing, aggregation, and pivoting on multiple levels.
Use `pd.MultiIndex.from_tuples()` or `set_index()` to define multiple index levels, enabling hierarchical access and manipulation of data.
- **`pd.IndexSlice`** is better for slicing when working with **MultiIndex** because it provides a clean and flexible way to specify slices across multiple levels of the index.

## 4. Data Cleaning and Preprocessing
Clean raw data by filling missing values, converting types, and removing duplicates.
- **Handling Missing Data**:
    - `.isna()`
    - `.fillna()`
    - `.dropna()`
    - `.interpolate()`: Fills missing values by estimating them using neighboring values.
- For nun-numeric data, it's ok to use **`None`** for missing values, but BP to use **`np.nan`** for numerical or mixed datasets

In [41]:
import numpy as np

df = pd.DataFrame({'A': [1, np.nan, 3], 'B': [4, 5, np.nan]})
print(df)

# Check for missing values
print("Missing values:\n", df.isna())
print("Total Number of Missing Values:\n", df.isna().sum())

# Fill missing values
df_filled = df.fillna(0)
print("\nAfter filling missing values with 0:\n", df_filled)

# Drop rows with missing values
df_dropped = df.dropna()
print("\nAfter dropping rows with missing values:\n", df_dropped)

# Interpolate missing values
df_interpolated = df.interpolate()
print("\nAfter interpolating missing values:\n", df_interpolated)


     A    B
0  1.0  4.0
1  NaN  5.0
2  3.0  NaN
Missing values:
        A      B
0  False  False
1   True  False
2  False   True
Total Number of Missing Values:
 A    1
B    1
dtype: int64

After filling missing values with 0:
      A    B
0  1.0  4.0
1  0.0  5.0
2  3.0  0.0

After dropping rows with missing values:
      A    B
0  1.0  4.0

After interpolating missing values:
      A    B
0  1.0  4.0
1  2.0  5.0
2  3.0  5.0


- **Data Type Conversion**: `.astype()`
- **String Operations**:`.str` methods for cleaning and manipulating text data. (.upper(), .strip(), .replace(), .contains(), .len())
- **Duplicated Data**:
    - **`.duplicated()`**: Flags duplicate rows.
    - **`.drop_duplicates()`**: Removes duplicates, keeping the first occurrence by default.

In [42]:
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Alice', 'Charlie', 'Bob'],
    'Age': [25, 30, 25, 35, 30]
})

# Converting data types
df['Age'] = df['Age'].astype(float)
print(df.dtypes)

# String operations
df['Name'] = df['Name'].str.upper()
print(df)

# Check for duplicated rows
duplicates = df.duplicated()
print("Duplicated rows:\n", duplicates)

# Drop duplicate rows
df_no_duplicates = df.drop_duplicates()
print("\nDataFrame after dropping duplicates:\n", df_no_duplicates)

Name     object
Age     float64
dtype: object
      Name   Age
0    ALICE  25.0
1      BOB  30.0
2    ALICE  25.0
3  CHARLIE  35.0
4      BOB  30.0
Duplicated rows:
 0    False
1    False
2     True
3    False
4     True
dtype: bool

DataFrame after dropping duplicates:
       Name   Age
0    ALICE  25.0
1      BOB  30.0
3  CHARLIE  35.0


## 5. Data Transformation
Transform datasets into desired shapes or formats for analysis.
- **Column Operations**:
    - Add, remove, rename, and reorder columns.
- **Apply and Map Functions**:
    - `.apply()`, `.map()`, `.applymap()` for custom transformations.
- **Sorting**:
    - `.sort_values()` / `.sort_index()`.
- **Pivot Tables**:
    - `.pivot_table()` for aggregations and summaries.
- **Melt and Reshape**:
    - `.melt()`, `.stack()`, `.unstack()`.

In [37]:
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'London', 'Paris']
}, index=['row1', 'row2', 'row3'])
print(df)

         Name  Age      City
row1    Alice   25  New York
row2      Bob   30    London
row3  Charlie   35     Paris


In [39]:
df['Salary'] = [50000, 60000, 70000] 
print("Added a Salary column:\n", df)

df['Salary'] = df['Salary'] * 1.1
print("\nIncrease Salary by 10%:\n", df)

df.rename(columns={'Salary': 'Income'}, inplace=True) 
print("\nRenamed Salary column to Income:\n", df)

df['Net_income'] = df['Income'].apply(lambda x: x * 0.7)
print("\n Create Net_income column, applied a lambda function:\n", df)

pivot = pd.pivot_table(df, values='Income', index='Age', aggfunc='mean')
print("\nPivot table (Mean income by Age):\n", pivot)

df = df.drop(columns=['Income'])
print("\nDeleted Income column:\n", df)

df['Net_income'].replace(38500.0, 40000.0, inplace=True)  # Replace 
print("\nReplace Alice's Net Income:\n", df)

Added a Salary column:
          Name  Age      City  Net_income  Salary
row1    Alice   25  New York     40000.0   50000
row2      Bob   30    London     46200.0   60000
row3  Charlie   35     Paris     53900.0   70000

Increase Salary by 10%:
          Name  Age      City  Net_income   Salary
row1    Alice   25  New York     40000.0  55000.0
row2      Bob   30    London     46200.0  66000.0
row3  Charlie   35     Paris     53900.0  77000.0

Renamed Salary column to Income:
          Name  Age      City  Net_income   Income
row1    Alice   25  New York     40000.0  55000.0
row2      Bob   30    London     46200.0  66000.0
row3  Charlie   35     Paris     53900.0  77000.0

 Create Net_income column, applied a lambda function:
          Name  Age      City  Net_income   Income
row1    Alice   25  New York     38500.0  55000.0
row2      Bob   30    London     46200.0  66000.0
row3  Charlie   35     Paris     53900.0  77000.0

Pivot table (Mean income by Age):
       Income
Age         
2

## 6. Aggregation and Grouping
**GroupBy Operations**: Group data by a column and calculate aggregate statistics.
 - `.groupby()` for aggregation (`sum`, `mean`, `count`, etc.).
 - **`.groupby('Col1', 'Col2').agg()`** applies multiple aggregation functions (sum, mean, min, max) on different columns (Col1, Col2) in a single operation.

In [46]:
df = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'A', 'B'],
    'Value1': [10, 20, 30, 40, 50, 60],
    'Value2': [5, 15, 25, 35, 45, 55]
})
print(df)

# Group by 'Category' and calculate sum of each column
grouped_sum = df.groupby('Category').sum()
print("GroupBy Sum:\n", grouped_sum)

# Group by 'Category' and count non-null entries in each column
grouped_count = df.groupby('Category').count()
print("\nGroupBy Count:\n", grouped_count)

# Group by 'Category' and calculate multiple aggregation statistics
grouped = df.groupby('Category').agg({
    'Value1': ['sum', 'mean'],    # Sum and mean of Value1
    'Value2': ['min', 'max']      # Min and max of Value2
})
print("GroupBy Aggregation using .agg():\n", grouped)

  Category  Value1  Value2
0        A      10       5
1        A      20      15
2        B      30      25
3        B      40      35
4        A      50      45
5        B      60      55
GroupBy Sum:
           Value1  Value2
Category                
A             80      65
B            130     115

GroupBy Count:
           Value1  Value2
Category                
A              3       3
B              3       3
GroupBy Aggregation using .agg():
          Value1            Value2    
            sum       mean    min max
Category                             
A            80  26.666667      5  45
B           130  43.333333     25  55


**Window and Rolling Operations**: Apply rolling metrics on time-series data 
- **`set_index()`**: Sets the specified column as the index of the DF
- **`rolling(window=3)`**: Creates a rolling window view of the data over a specified window size-> can calculate metrics like `mean()`, `min()`, `max()`,  `std()`,  `var()`,  `sum()` within that window. Can also use `count()`and `apply()` (for custom functions).
- **`expanding()`**: Expands the window to include all prior data points for each row, allowing for cumulative calculations.

In [44]:
df = pd.DataFrame({
    'Date': pd.date_range('2023-01-01', periods=6, freq='D'),
    'Value': [10, 20, 30, 40, 50, 60]
})
print(df)

# Set Date as index
df.set_index('Date', inplace=True)

# Apply rolling window operation (e.g., 3-day rolling sum)
df['Rolling_Sum'] = df['Value'].rolling(window=3).sum()

# Apply expanding window operation (cumulative sum)
df['Expanding_Sum'] = df['Value'].expanding().sum()

print("\nRolling and Expanding Operations:\n", df)

        Date  Value
0 2023-01-01     10
1 2023-01-02     20
2 2023-01-03     30
3 2023-01-04     40
4 2023-01-05     50
5 2023-01-06     60

Rolling and Expanding Operations:
             Value  Rolling_Sum  Expanding_Sum
Date                                         
2023-01-01     10          NaN           10.0
2023-01-02     20          NaN           30.0
2023-01-03     30         60.0           60.0
2023-01-04     40         90.0          100.0
2023-01-05     50        120.0          150.0
2023-01-06     60        150.0          210.0


## 7. Merging and Combining Data
Combine data from multiple sources into a single coherent dataset.
- **Merge and Join**:
    - `.merge()` (similar to SQL joins).
    - `.join()` for index-based merging.
- **Concatenation**:
    - `pd.concat()` to combine along a specific axis.
- **Database-like Operations**:
    - Append, union, intersection.

In [ ]:
# Create two DataFrames
df1 = pd.DataFrame({'ID': [1, 2], 'Name': ['Alice', 'Bob']})
df2 = pd.DataFrame({'ID': [2, 3], 'City': ['London', 'Paris']})

# Merge DataFrames
merged = pd.merge(df1, df2, on='ID', how='outer')
print(merged)


# Concatenating DataFrames
df3 = pd.DataFrame({'Name': ['Eve'], 'Age': [40], 'Income': [80000]})
concat = pd.concat([df, df3], ignore_index=True)
print(concat)

## 8. Time-Series Data
Handle and analyze temporal data efficiently.
- **Datetime Handling**:
    - `pd.to_datetime()`, `.dt` accessor for datetime attributes.
- **Resampling**:
    - `.resample()` for up/down sampling of time-series data.
- **Shifting and Lagging**:
    - `.shift()` for lag analysis.

In [ ]:
# Creating a time-series DataFrame
time_data = pd.date_range(start='2023-01-01', periods=5, freq='D')
time_df = pd.DataFrame({'Date': time_data, 'Value': range(5)})
time_df.set_index('Date', inplace=True)
print(time_df)

# Resampling data
resampled = time_df.resample('2D').sum()
print(resampled)

## 9. Performance Optimization
Optimize memory and computational performance.
- **Memory Optimization**:
    - Reducing data size by converting types (`pd.to_numeric()` with `downcast`).
- **Vectorization**:
    - Leveraging NumPy-backed operations over Python loops.
- **Efficient I/O**:
    - Use binary formats like Parquet or Feather for faster reads/writes.

In [ ]:
# Reducing memory usage by converting data types
optimized_df = df.copy()
optimized_df['Income'] = pd.to_numeric(optimized_df['Income'], downcast='integer')
print(optimized_df.dtypes)

# Using vectorized operations
df['Adjusted_Income'] = df['Income'] * 0.9
print(df)

## 10. Visualization
Quickly visualize data trends and distributions.
- **Basic Plotting**:
    - `.plot()` for quick visualizations.
- **Integration with Libraries**:
    - Seamless use with `matplotlib`, `seaborn`, and `plotly`.

In [ ]:
# Basic plotting
df['Income'].plot(kind='bar')

## 11. Advanced Topics
Master advanced operations like MultiIndex, sparse matrices, custom aggregations, and handling hierarchical data.
- **Custom Aggregations**:
    - Using `.agg()` with lambda functions or multiple aggregations per column.
- **MultiIndex Operations**:
    - Slicing, swapping levels, and combining MultiIndex DataFrames.
- **Categorical Data**:
    - Optimizing memory and performance with `.astype('category')`.
- **Sparse Data**:
    - Handling sparse DataFrames for memory efficiency.
- **Working with JSON/Hierarchical Data**:
    - Normalizing nested JSON with `pd.json_normalize()`.

In [ ]:
# MultiIndex DataFrame
multi_index_df = pd.DataFrame({'Category': ['A', 'A', 'B'], 'Subcategory': ['X', 'Y', 'Z'], 'Values': [1, 2, 3]})
multi_index_df.set_index(['Category', 'Subcategory'], inplace=True)
print(multi_index_df)

# Custom aggregations
custom_agg = df.agg({'Income': ['sum', 'mean'], 'Age': 'max'})
print(custom_agg)